In [1]:
%run 2021_11_02_Psilocybin_in_Cg_Exchange_Function.ipynb #change model source here

No objective coefficients in model. Unclear what should be optimized


In [2]:
model.reactions.EX_psi_e

Reaction identifier,EX_psi_e
Name,Psilocybin_e exchange
Memory address,0x026a28a63470
Stoichiometry,psi_e <=> Psilocybin_e <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [3]:
with model:
    model.objective = model.reactions.EX_psi_e
    print(model.optimize().objective_value)

0.0


# Overview of Reactions

In [4]:
model.metabolites.trp_L_c

Metabolite identifier,trp_L_c
Name,L_Tryptophan
Memory address,0x026a40a83320
Formula,C11H12N2O2
Compartment,c
In 7 reaction(s),"TRPS1, TRPS2, TRPt2rpp, TRPabcpp, PROTEIN, TRPTRS, CrTdc"


In [5]:
model.metabolites.trp_L_e

Metabolite identifier,trp_L_e
Name,L_Tryptophan
Memory address,0x026a40a83390
Formula,C11H12N2O2
Compartment,e
In 3 reaction(s),"EX_trp_L_e, TRPabcpp, TRPt2rpp"


In [6]:
model.reactions.TRPt2rpp

Reaction identifier,TRPt2rpp
Name,L_tryptophan reversible transport via proton symport
Memory address,0x026a412bc940
Stoichiometry,h_e + trp_L_e <-- h_c + trp_L_c H+ + L_Tryptophan <-- H+ + L_Tryptophan
GPR,cg1257
Lower bound,-77.3942899999992
Upper bound,-1.83256065354456e-29


In [7]:
model.reactions.TRPabcpp

Reaction identifier,TRPabcpp
Name,L_trp transport via ABC system
Memory address,0x026a412bc748
Stoichiometry,atp_c + h2o_c + trp_L_e --> adp_c + h_c + pi_c + trp_L_c ATP + H2O + L_Tryptophan --> ADP + H+ + Phosphate + L_Tryptophan
GPR,
Lower bound,0.0
Upper bound,77.3942899999993


In [8]:
model.reactions.CrTdc

Reaction identifier,CrTdc
Name,
Memory address,0x026a410d1fd0
Stoichiometry,trp_L_c --> co2_c + tryp_c L_Tryptophan --> CO2 + tryptamine
GPR,
Lower bound,0.0
Upper bound,1000.0


In [9]:
model.reactions.psiH

Reaction identifier,psiH
Name,
Memory address,0x026a410d1ac8
Stoichiometry,2 nadph_c + o2_c + tryp_c --> _4htryp_c + h2o_c + 2 nadp_c 2 Nicotinamide adenine dinucleotide phosphate - reduced + O2 + tryptamine --> 4-Hydroxytryptamine + H2O + 2 Nicotinamide adenine dinucleotide phosphate
GPR,
Lower bound,0.0
Upper bound,1000.0


In [10]:
model.reactions.psiK

Reaction identifier,psiK
Name,
Memory address,0x026a410d1eb8
Stoichiometry,_4htryp_c + atp_c --> adp_c + nbc_c 4-Hydroxytryptamine + ATP --> ADP + Norbaeocystin
GPR,
Lower bound,0.0
Upper bound,1000.0


In [11]:
model.reactions.psiM

Reaction identifier,psiM
Name,
Memory address,0x026a410d1e48
Stoichiometry,amet_c + nbc_c --> ahcys_c + baecyst_c S-AdenosyL_L_methionine + Norbaeocystin --> S-AdenosyL_L_homocysteine + Baeocystin
GPR,
Lower bound,0.0
Upper bound,1000.0


In [12]:
model.reactions.psiM2

Reaction identifier,psiM2
Name,
Memory address,0x026a410d1d30
Stoichiometry,amet_c + baecyst_c --> ahcys_c + psi_c S-AdenosyL_L_methionine + Baeocystin --> S-AdenosyL_L_homocysteine + Psilocybin
GPR,
Lower bound,0.0
Upper bound,1000.0


In [14]:
#model.reactions.psiK2

In [16]:
#model.reactions.pho

# Set Bounds Here

In [30]:
#model.reactions.TRPS1.upper_bound=100 #ensure tryptophan production, explore media later. 
#model.reactions.TRPS1.lower_bound=.1

model.reactions.TRPt2rpp.upper_bound=0 #Knock out tryptophan exporter.
model.reactions.TRPt2rpp.lower_bound=0

In [31]:
model.reactions.TRPS1

Reaction identifier,TRPS1
Name,tryptophan synthase (indoleglycerol phosphate)
Memory address,0x026a41335908
Stoichiometry,3ig3p_c + ser_L_c --> g3p_c + h2o_c + trp_L_c C'-(3-Indolyl)-glycerol 3-phosphate + L_Serine --> Glyceraldehyde 3-phosphate + H2O + L_Tryptophan
GPR,cg3363 and cg3364
Lower bound,0.1
Upper bound,100


# Resulting Flux Optimized for Psi Production

In [32]:
with model:
    model.objective = model.reactions.EX_psi_e
    print("every reaction going toward psilocybin")
    
    print("Max. psi production [mmol gDW^-1 h^-1]:",model.optimize().objective_value)
    print("tryptamine [mmol gDW^-1 h^-1]:",model.reactions.CrTdc.flux)
    print("4-Hydroxytryptamine [mmol gDW^-1 h^-1]:",model.reactions.psiH.flux)
    print("")
    

every reaction going toward psilocybin
Max. psi production [mmol gDW^-1 h^-1]: 0.08776019008215959
tryptamine [mmol gDW^-1 h^-1]: 0.0
4-Hydroxytryptamine [mmol gDW^-1 h^-1]: 0.0



In [35]:
model.medium

{'EX_ca2_e': 0.0144877134286247,
 'EX_cobalt2_e': 7.00338234344926e-05,
 'EX_cu2_e': 0.00197271722237563,
 'EX_fe2_e': 0.16239125,
 'EX_fe3_e': 0.0217300637039616,
 'EX_glc_e': 4.67,
 'EX_k_e': 0.543285250115532,
 'EX_mg2_e': 0.0241459221505149,
 'EX_mn2_e': 0.00192443552016793,
 'EX_nh4_e': 11.9121416666662,
 'EX_ni2_e': 0.00089824250292982,
 'EX_o2_e': 16.5067783333334,
 'EX_pi_e': 0.918000000000888,
 'EX_so4_e': 4.01673156249995,
 'EX_zn2_e': 0.000948926279874219,
 'EX_BIOTIN': 1.01925397503027e-06,
 'EX_cl': 0.0144877134286247,
 'EX_psi_e': 1000.0}

In [33]:
with model:
    model.objective = model.reactions.EX_psi_e
    print("every reaction going from Typtophan")
    
    print("Max. psi production [mmol gDW^-1 h^-1]:",model.optimize().objective_value)
    print("TRPTRS [mmol gDW^-1 h^-1]:",model.reactions.TRPTRS.flux)
    print("TRPS2 [mmol gDW^-1 h^-1]:",model.reactions.TRPS2.flux)
    print("CrTdc [mmol gDW^-1 h^-1]:",model.reactions.CrTdc.flux)
    print("TRPS1 [mmol gDW^-1 h^-1]:",model.reactions.TRPS1.flux)
    print("TRPt2rpp [mmol gDW^-1 h^-1]:",model.reactions.TRPt2rpp.flux)
    print("TRPabcpp [mmol gDW^-1 h^-1]:",model.reactions.TRPabcpp.flux)
    print("PROTEIN [mmol gDW^-1 h^-1]:",model.reactions.PROTEIN.flux)
    
    

every reaction going from Typtophan
Max. psi production [mmol gDW^-1 h^-1]: 0.08776019008215959
TRPTRS [mmol gDW^-1 h^-1]: 0.0
TRPS2 [mmol gDW^-1 h^-1]: 0.0
CrTdc [mmol gDW^-1 h^-1]: 0.0
TRPS1 [mmol gDW^-1 h^-1]: 0.1
TRPt2rpp [mmol gDW^-1 h^-1]: -0.08776019008215953
TRPabcpp [mmol gDW^-1 h^-1]: 0.0
PROTEIN [mmol gDW^-1 h^-1]: 0.21896294956690587


not viable to knock out protein production

In [28]:
model.reactions.TRPt2rpp

Reaction identifier,TRPt2rpp
Name,L_tryptophan reversible transport via proton symport
Memory address,0x026a412bc940
Stoichiometry,h_e + trp_L_e --> h_c + trp_L_c H+ + L_Tryptophan --> H+ + L_Tryptophan
GPR,cg1257
Lower bound,0
Upper bound,0
